aqui especificar el ETL y hago los requerimientos que me piden en el readme del github... acomodar introduccion

In [112]:
# Importar las bibliotecas necesarias
import numpy as np
import pandas as pd
import json
import ast

In [113]:
#Extraer el dataset
df_movies = pd.read_csv('Dataset/movies_dataset.csv', encoding='utf-8')

C:\Users\evely\AppData\Local\Temp\ipykernel_16400\3148652920.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('Dataset/movies_dataset.csv', encoding='utf-8')


In [114]:

print(df_movies.shape)
df_movies.head(1)

(45466, 24)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [115]:

# Crear una copia del DataFrame original para no modificarlo directamente
df = df_movies.copy()


In [116]:
# Mostrar solo la columna 'belongs_to_collection'
print(df['belongs_to_collection'])

0        {'id': 10194, 'name': 'Toy Story Collection', ...
1                                                      NaN
2        {'id': 119050, 'name': 'Grumpy Old Men Collect...
3                                                      NaN
4        {'id': 96871, 'name': 'Father of the Bride Col...
                               ...                        
45461                                                  NaN
45462                                                  NaN
45463                                                  NaN
45464                                                  NaN
45465                                                  NaN
Name: belongs_to_collection, Length: 45466, dtype: object


In [117]:

# Función para desanidar los diccionarios en la columna "belongs_to_collection"
def desanidar_diccionario(value):
    if pd.isnull(value):
        return {}
    try:
        return ast.literal_eval(value)
    except (ValueError, SyntaxError):
        return {}

# Desanidar y expandir la columna en un nuevo DataFrame
expanded_col = df['belongs_to_collection'].apply(desanidar_diccionario).apply(pd.Series)

# Renombrar la columna 'id' a 'idcollection' si existe
if 'id' in expanded_col.columns:
    expanded_col.rename(columns={'id': 'idcollection'}, inplace=True)

# Eliminar columnas con todos los valores NaN
expanded_col = expanded_col.dropna(axis=1, how='all')

# Concatenar el DataFrame original con el DataFrame expandido
movies = pd.concat([df.drop(columns=['belongs_to_collection']), expanded_col], axis=1)

# Ver los primeros registros
print(movies.head())


   adult    budget                                             genres  \
0  False  30000000  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  False  65000000  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  False         0  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  False  16000000  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4  False         0                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                   NaN   8844  tt0113497                en   
2                                   NaN  15602  tt0113228                en   
3                                   NaN  31357  tt0114885                en   
4                                   NaN  11862  tt0113041                en   

                original_title  \
0                    Toy Story   
1                 

In [118]:
movies.head(1)

,adult,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,tagline,title,video,vote_average,vote_count,idcollection,name,poster_path,backdrop_path,0
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,NaN,Toy Story,False,7.7,5415.0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,NaN


Ahora se procede a desanidar las demas columnas 

In [119]:
# Desanidar columnas del dataframe 

# Lista de columnas anidadas a desanidar
columnas_anidadas = ['genres', 'production_companies', 'production_countries', 'spoken_languages']

# Función para desanidar columnas que contienen listas de diccionarios
def desanidar_columna(df, columna):
    # Convertir cadenas de texto en listas/diccionarios si es necesario
    df[columna] = df[columna].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    
    # Desanidar según el tipo de datos en la columna
    if df[columna].apply(lambda x: isinstance(x, list)).any():
        # Para columnas con listas de diccionarios
        desanidado = pd.json_normalize(df[columna].explode().dropna()).reset_index(drop=True)
    else:
        # Para columnas con diccionarios simples
        desanidado = pd.json_normalize(df[columna].dropna()).reset_index(drop=True)
    
    # Renombrar las columnas desanidadas
    desanidado.columns = [f"{columna}_{subcol}" for subcol in desanidado.columns]
    
    # Unir las columnas desanidadas al DataFrame original
    df = df.drop(columns=[columna]).join(desanidado)
    return df

# Desanidar columnas que contienen listas de diccionarios
for columna in ['genres', 'production_companies', 'production_countries', 'spoken_languages']:
    movies = desanidar_columna(movies, columna)


In [120]:
# verificar columnas desanidadas
print(movies.shape)
movies.head(1)


(45466, 32)


,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,backdrop_path,0,genres_id,genres_name,production_companies_name,production_companies_id,production_countries_iso_3166_1,production_countries_name,spoken_languages_iso_639_1,spoken_languages_name
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,NaN,16,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English


In [121]:
#IMPRIMIR TODAS LAS COLUMNAS
nombres_columnas = movies.columns
print(nombres_columnas)


Index([                          'adult',                          'budget',
                              'homepage',                              'id',
                               'imdb_id',               'original_language',
                        'original_title',                        'overview',
                            'popularity',                     'poster_path',
                          'release_date',                         'revenue',
                               'runtime',                          'status',
                               'tagline',                           'title',
                                 'video',                    'vote_average',
                            'vote_count',                    'idcollection',
                                  'name',                     'poster_path',
                         'backdrop_path',                                 0,
                             'genres_id',                     'genres_name',

Se deben eliminar columnas de las desanidadas que no se usaran, las solicitadas y otras que no tienen relevancia en el modelado

In [122]:
# Eliminar las columnas que no serán utilizadas
columns_to_drop = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage', 'backdrop_path', 'genres_id',
                   'production_countries_iso_3166_1', 'spoken_languages_iso_639_1','idcollection','production_companies_id',0]
movies = movies.drop(columns=columns_to_drop)

In [123]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   budget                     45466 non-null  object 
 1   id                         45466 non-null  object 
 2   original_language          45455 non-null  object 
 3   overview                   44512 non-null  object 
 4   popularity                 45461 non-null  object 
 5   release_date               45379 non-null  object 
 6   revenue                    45460 non-null  float64
 7   runtime                    45203 non-null  float64
 8   status                     45379 non-null  object 
 9   tagline                    20412 non-null  object 
 10  title                      45460 non-null  object 
 11  vote_average               45460 non-null  float64
 12  vote_count                 45460 non-null  float64
 13  name                       4491 non-null   obj

In [125]:

# Renombrar algunas columnas
movies = movies.rename(columns={
    'id': 'idMovies',
    'name': 'nameColeccion',
    'genres_name': 'genres',
    'production_companies_name': 'production_companies',
    'production_countries_name': 'production_countries',
    'spoken_languages_name': 'spoken_language'

})

# Verificar las primeras filas del DataFrame para asegurarse de que las columnas han sido renombradas
movies.head(1)

,budget,idMovies,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,nameColeccion,genres,production_companies,production_countries,spoken_language
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,Animation,Pixar Animation Studios,United States of America,English


In [126]:
#mostrar solo las 5 primeras filas
print(movies.shape)
movies.head(5)

(45466, 18)


,budget,idMovies,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,nameColeccion,genres,production_companies,production_countries,spoken_language
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,Animation,Pixar Animation Studios,United States of America,English
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,NaN,Comedy,TriStar Pictures,United States of America,English
2,0,15602,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,Family,Teitler Film,United States of America,Français
3,16000000,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,NaN,Adventure,Interscope Communications,United States of America,English
4,0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,Father of the Bride Collection,Fantasy,Warner Bros.,United States of America,English


Transformaciones solicitadas

In [127]:
# Rellenar los valores nulos en las columnas 'revenue' y 'budget' con 0

movies['revenue'] = movies['revenue'].fillna(0)
movies['budget'] = movies['budget'].fillna(0)

# Ver los primeros registros para verificar
print(movies[['revenue', 'budget']].head())


       revenue    budget
0  373554033.0  30000000
1  262797249.0  65000000
2          0.0         0
3   81452156.0  16000000
4   76578911.0         0


In [128]:
# Eliminar las filas donde el campo 'release date' es nulo
movies = movies.dropna(subset=['release_date'])
print(movies[['release_date']].head())

  release_date
0   1995-10-30
1   1995-12-15
2   1995-12-22
3   1995-12-22
4   1995-02-10


In [130]:
# Convertir la columna 'release_date' al formato de fecha AAAA-mm-dd
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce', format='%Y-%m-%d')

# Crear la columna 'release_year' extrayendo el año de 'release_date' y convirtiendo a formato de fecha con frecuencia anual
movies['release_year'] = movies['release_date'].dt.to_period('Y')

# Opcional: Eliminar filas con 'release_year' nulo
movies = movies.dropna(subset=['release_year'])

print(movies[['release_date', 'release_year']].head())


  release_date release_year
0   1995-10-30         1995
1   1995-12-15         1995
2   1995-12-22         1995
3   1995-12-22         1995
4   1995-02-10         1995


In [131]:
#Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, 
# dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0

# Asegurarse de que las columnas 'revenue' y 'budget' sean numéricas
movies['revenue'] = pd.to_numeric(movies['revenue'], errors='coerce')
movies['budget'] = pd.to_numeric(movies['budget'], errors='coerce')

# Crear la columna 'return' calculando revenue / budget
# Si el budget es 0 o NaN, el retorno también debe ser 0
movies['return'] = movies.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] > 0 else 0, axis=1)

# Verificar los primeros registros para asegurarse de que el cálculo es correcto
print(movies[['revenue', 'budget', 'return']].head())


       revenue    budget     return
0  373554033.0  30000000  12.451801
1  262797249.0  65000000   4.043035
2          0.0         0   0.000000
3   81452156.0  16000000   5.090760
4   76578911.0         0   0.000000


Realizar otras modificaciones necesarias

In [132]:
#Verificacion de los tipos de datos
print(movies.dtypes)

budget                           int64
idMovies                        object
original_language               object
overview                        object
popularity                      object
release_date            datetime64[ns]
revenue                        float64
runtime                        float64
status                          object
tagline                         object
title                           object
vote_average                   float64
vote_count                     float64
nameColeccion                   object
genres                          object
production_companies            object
production_countries            object
spoken_language                 object
release_year             period[Y-DEC]
return                         float64
dtype: object


In [133]:
# Modificación de idMovies
# Convertir la columna 'id' a entero
movies['idMovies'] = movies['idMovies'].astype(int)


In [134]:
# Convertir 'popularity' a float64
movies['popularity'] = movies['popularity'].astype(float)

# Verificar los tipos de datos después de las conversiones
print(movies.dtypes)

budget                           int64
idMovies                         int32
original_language               object
overview                        object
popularity                     float64
release_date            datetime64[ns]
revenue                        float64
runtime                        float64
status                          object
tagline                         object
title                           object
vote_average                   float64
vote_count                     float64
nameColeccion                   object
genres                          object
production_companies            object
production_countries            object
spoken_language                 object
release_year             period[Y-DEC]
return                         float64
dtype: object


Verificacion de Campos Nulos

In [135]:
#valores nulos
movies.isnull().sum()

budget                      0
idMovies                    0
original_language          11
overview                  941
popularity                  0
release_date                0
revenue                     0
runtime                   246
status                     80
tagline                 24978
title                       0
vote_average                0
vote_count                  0
nameColeccion           40888
genres                      0
production_companies        1
production_countries        3
spoken_language             0
release_year                0
return                      0
dtype: int64

In [138]:
print(movies['nameColeccion'].unique())

['Toy Story Collection' nan 'Grumpy Old Men Collection' ...
 'Ducobu Collection' 'Mister Blot Collection' 'Red Lotus Collection']


Resumen: Eliminación de la Columna coleccion_de_peliculas ya que posee como el 90% de valores nulos que pueden afectar en mi modelo

In [140]:
# Eliminar las columnas que no serán utilizadas
columns_to_drop = 'nameColeccion'
movies = movies.drop(columns=columns_to_drop)

In [141]:
#valores nulos
movies.isnull().sum()

budget                      0
idMovies                    0
original_language          11
overview                  941
popularity                  0
release_date                0
revenue                     0
runtime                   246
status                     80
tagline                 24978
title                       0
vote_average                0
vote_count                  0
genres                      0
production_companies        1
production_countries        3
spoken_language             0
release_year                0
return                      0
dtype: int64

In [142]:
# rellenar columnas para no eliminar columnas quizas necesarias 
fill_values = {
    'original_language': 'unknown',
    'overview': 'no summary',
    'tagline': 'no summary',
    'status': 'unknown',
    'compañiaProduccion': 'unknown',
    'paisProduccion': 'unknown',
    'runtime': 0
    
}

# Rellenar los valores nulos en las columnas especificadas usando un diccionario
movies.fillna(value=fill_values, inplace=True)

In [1]:
#verificar tamaño
movies.shape()


NameError: name 'movies' is not defined

In [144]:
#valores nulos
movies.isnull().sum()

budget                  0
idMovies                0
original_language       0
overview                0
popularity              0
release_date            0
revenue                 0
runtime                 0
status                  0
tagline                 0
title                   0
vote_average            0
vote_count              0
genres                  0
production_companies    1
production_countries    3
spoken_language         0
release_year            0
return                  0
dtype: int64

In [145]:

# Exportar los cambios realizados 
movies.to_csv('Dataset/movies_final.csv', index=False)
